In [ ]:
import numpy as np
import sep
import matplotlib.pyplot as plt
import astropy

from astropy.io import fits
from matplotlib import rcParams

%matplotlib inline

rcParams['figure.figsize'] = [10., 8.]

save_path = '/Users/labuser/Desktop/'

In [ ]:
filename = "/Users/labuser/Downloads/hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits" 
hdu_list = fits.open(filename) 
hdu_list.info() 
image_data = hdu_list[0].data

In [ ]:
image_data = hdu_list[0].data

m, s = np.mean(image_data), np.std(image_data)
plt.imshow(image_data, interpolation = 'nearest', cmap='gray', vmin = m-s, vmax = m+s, origin='lower')
plt.colorbar()

#plt.savefig(save_path + 'raw_image.png', dpi = 600)

# Background subtraction

In [ ]:
#Input array with dtype '>f4' has non-native byte order. Only native byte order arrays are supported. To change the byte order of the array 'data'
image_data = image_data.byteswap().newbyteorder()


bkg = sep.Background(image_data)
bkg = sep.Background(image_data, bw=64, bh=64, fw=3, fh=3)

print(bkg.globalback)
print(bkg.globalrms)

In [ ]:
bkg_image = bkg.back()

plt.imshow(bkg_image, interpolation = 'nearest', cmap = 'gray', origin = 'lower')
plt.colorbar();

#plt.savefig(save_path + 'background.png', dpi = 600)

In [ ]:
bkg_rms = bkg.rms()

plt.imshow(bkg_rms, interpolation = 'nearest', cmap = 'gray', origin = 'lower')
plt.colorbar();

#plt.savefig(save_path + 'background_noise.png', dpi = 600)

In [ ]:
# subtract the background
data_sub = image_data - bkg

# Object detection

In [ ]:
objects = sep.extract(data_sub, 1.5, err = bkg.globalrms)

len(objects)

In [ ]:
from matplotlib.patches import Ellipse    #Ellipse fitting

#plot bagkround_subtracted image
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation = 'nearest', cmap = 'gray', vmin = m - s, vmax = m + s, origin = 'lower')

#plot an ellipse for each object
#objects['x'] and objects['y'] will give the centroid coordinates of the objects.
for i in range(len(objects)):
    e = Ellipse(xy = (objects['x'][i], objects['y'][i]), width = 6*objects['a'][i], height = 6*objects['b'][i], angle = objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')   #leaves the center of the ellipse clear
    e.set_edgecolor('red')    #outlines the ellipse with a specified color
    ax.add_artist(e)          #add the ellipse to the image

#Save the background at a high resolution.
plt.savefig(save_path + 'f105w_sources_detection.png', dpi = 600)

# Aperture photometry

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'], 3.0, err = bkg.globalrms, gain = 1.0)

In [ ]:
for i in range(len(objects)):
    print("object {:d}: flux = {:f} +/- {:f}".format(i + 1, flux[i], fluxerr[i]))

# Convert to AB Magnitude

In [ ]:
AB_mag = [0] * len(objects)
for nmbr in range(len(objects)):
    
    AB_mag[nmbr]= -2.5*np.log10(flux[nmbr]) - 48.60
    print(AB_mag[nmbr])

# Histogram

In [ ]:
histogram = plt.hist(AB_mag, bins=len(AB_mag))
plt.xlim(-55,-39)
plt.ylim(0,15)